In [ ]:
import numpy as np, torch, pickle, feedparser, requests, sqlite3, re
from datetime import datetime
from pathlib import Path
from typing import Union, List, Dict, Tuple, Optional, Callable, Set
from dataclasses import dataclass
from fastcore.utils import *
from claudette import *

@dataclass
class RiskProfile:
    name: str
    ticker: Optional[str] = None
    industry: Optional[str] = None
    gics_code: Optional[str] = None
    risk_factors: Set[str] = None
    
    def __post_init__(self):
        if self.risk_factors is None: self.risk_factors = set()
        self.risk_factors.add(self.name.lower())
        if self.ticker: self.risk_factors.add(self.ticker.lower())
        if self.industry: self.risk_factors.add(self.industry.lower())

class RSScraper:
    def __init__(self, feeds): self.feeds = feeds if isinstance(feeds, list) else [feeds]
    def fetch_feed(self, url): return feedparser.parse(url)
    def get_articles(self, limit=50):
        articles = []
        for feed_url in self.feeds:
            feed = self.fetch_feed(feed_url)
            for entry in feed.entries[:limit]: articles.append(self._parse_entry(entry))
        return articles[:limit]
    def _parse_entry(self, entry): 
        summary = entry.get('summary', '') or entry.get('description', '')
        return dict(title=entry.get('title', ''), link=entry.get('link', ''), published=entry.get('published', ''), summary=summary)
    # def _parse_entry(self, entry): return dict(title=entry.get('title', ''), link=entry.get('link', ''), published=entry.get('published', ''), summary=entry.get('summary', ''))

@patch
def analyze_sentiment_ai(self:RSScraper, text, risk_profile=None):
    sp = """You are a financial risk analyst. Analyze sentiment and risk impact on a scale of -10 to +10 where:
    -10: Extremely negative (major crisis, bankruptcy, massive losses)
    -5: Negative (earnings miss, regulatory issues, downgrades)
    0: Neutral (routine news, mixed signals)
    +5: Positive (earnings beat, upgrades, partnerships)
    +10: Extremely positive (breakthrough products, major acquisitions)
    
    Consider: market sentiment, fundamental impact, volatility implications, sector effects.
    Respond with only the number."""
    
    context = f"Risk factors: {', '.join(risk_profile.risk_factors) if risk_profile else 'general market'}"
    prompt = f"{context}\n\nNews: {text[:800]}"
    
    try:
        chat = Chat(models[1], sp=sp)
        result = chat(prompt)
        score = int(result.content[0].text.strip())
        return max(-10, min(10, score))
    except: return 0

@patch
def score_risk_impact_comprehensive(self:RSScraper, article, risk_profiles):
    text = f"{article['title']} {article['summary']}"
    risk_scores = []
    
    for profile in risk_profiles:
        base_score = 0
        text_lower = text.lower()
        
        if profile.name.lower() in text_lower: base_score += 5
        if profile.ticker and profile.ticker.lower() in text_lower: base_score += 3
        for factor in profile.risk_factors:
            if factor in text_lower: base_score += 1
        
        if base_score > 0:
            sentiment_score = self.analyze_sentiment_ai(text, profile)
            risk_contribution = abs(sentiment_score) * 0.1
            final_score = min(int(base_score * (1 + risk_contribution)), 10)
            if final_score > 0: risk_scores.append((profile, final_score, sentiment_score))
    
    return sorted(risk_scores, key=lambda x: x[1], reverse=True)

class NewsPrioritizer:
    def __init__(self, scraper, profiles): self.scraper,self.profiles = scraper,profiles
    def get_prioritized_news_ai(self, limit=10):
        articles = self.scraper.get_articles(limit)
        prioritized = []
        for article in articles:
            scores = self.scraper.score_risk_impact_comprehensive(article, self.profiles)
            max_score = scores[0][1] if scores else 0
            prioritized.append((article, max_score, scores))
        return sorted(prioritized, key=lambda x: x[1], reverse=True)
    def show_dashboard_ai(self, limit=5):
        news = self.get_prioritized_news_ai(limit)
        for i, (article, score, matches) in enumerate(news):
            if score > 0:
                print(f"\n{i+1}. PRIORITY {score}/10: {article['title']}")
                print(f"   Published: {article['published']}")
                for profile, pscore, sentiment in matches: print(f"   Impact: {profile.name} ({pscore}/10, sentiment: {sentiment})")
                print(f"   Link: {article['link']}")

financial_rss_feeds = ["https://feeds.bloomberg.com/businessweek/news.rss", "https://fortune.com/feed"]

finance_profiles = [
    RiskProfile("Tesla", "TSLA", "Automobiles", "251020", {"tesla", "tsla", "elon musk", "cybertruck", "electric vehicle", "ev", "autopilot", "robotaxi", "delivery", "production", "gigafactory"}),
    RiskProfile("Microsoft", "MSFT", "Software", "451030", {"microsoft", "msft", "ai", "artificial intelligence", "cloud", "azure", "pakistan", "operations", "diagnostic", "mai-dxo"}),
    RiskProfile("Apple", "AAPL", "Technology Hardware", "452020", {"apple", "aapl", "iphone", "earnings", "foxconn", "taiwan", "assembler"}),
    RiskProfile("Amazon", "AMZN", "Internet Retail", "255020", {"amazon", "amzn", "prime day", "shopping", "exec", "insider", "ai", "shein", "temu", "robotics"}),
    RiskProfile("Google/Alphabet", "GOOGL", "Internet Software", "451010", {"google", "googl", "alphabet", "nuclear", "fusion", "power", "virginia", "energy", "lawmakers"}),
    RiskProfile("Netflix", "NFLX", "Entertainment", "254010", {"netflix", "nflx", "streaming", "trillion", "market cap", "analyst", "upgrade"})
]

# finance_scraper = RSScraper(financial_rss_feeds)
# finance_prioritizer = NewsPrioritizer(finance_scraper, finance_profiles)
# finance_prioritizer.show_dashboard_ai(5)


In [ ]:
finance_scraper = RSScraper(financial_rss_feeds)

In [ ]:
finance_scraper

In [ ]:
finance_scraper.get_articles()

[{'title': '‘Telecom Is the New Tequila’:\xa0Behind the Celebrity Wireless\xa0Boom',
  'link': 'https://www.bloomberg.com/news/features/2025-07-07/why-the-trump-brothers-ryan-reynolds-became-telecom-entrepreneurs',
  'published': 'Mon, 07 Jul 2025 14:23:47 GMT',
  'summary': '<p>How Ryan Reynolds, the&nbsp;Trump brothers&nbsp;and other big names leveraged&nbsp;their fame to launch mobile services.</p>'},
 {'title': 'AI Turns\xa0Utility\xa0Analysts Into Rock Stars',
  'link': 'https://www.bloomberg.com/news/newsletters/2025-07-07/ai-s-demand-for-power-makes-utility-analysts-popular-among-investors',
  'published': 'Mon, 07 Jul 2025 16:45:20 GMT',
  'summary': '<p>Plus: 10 companies to watch this quarter, and why celebrity cellphone pitches keep coming.</p>'},
 {'title': 'Banking, Booze and 007’s Wheels: 10\xa0Companies to Watch Right Now',
  'link': 'https://www.bloomberg.com/features/10-companies-to-watch-q3-2025/',
  'published': 'Mon, 07 Jul 2025 04:01:20 GMT',
  'summary': 'Analysts

In [ ]:
L(finance_scraper.get_articles())


(#13) [{'title': '‘Telecom Is the New Tequila’:\xa0Behind the Celebrity Wireless\xa0Boom', 'link': 'https://www.bloomberg.com/news/features/2025-07-07/why-the-trump-brothers-ryan-reynolds-became-telecom-entrepreneurs', 'published': 'Mon, 07 Jul 2025 14:23:47 GMT', 'summary': '<p>How Ryan Reynolds, the&nbsp;Trump brothers&nbsp;and other big names leveraged&nbsp;their fame to launch mobile services.</p>'},{'title': 'AI Turns\xa0Utility\xa0Analysts Into Rock Stars', 'link': 'https://www.bloomberg.com/news/newsletters/2025-07-07/ai-s-demand-for-power-makes-utility-analysts-popular-among-investors', 'published': 'Mon, 07 Jul 2025 16:45:20 GMT', 'summary': '<p>Plus: 10 companies to watch this quarter, and why celebrity cellphone pitches keep coming.</p>'},{'title': 'Banking, Booze and 007’s Wheels: 10\xa0Companies to Watch Right Now', 'link': 'https://www.bloomberg.com/features/10-companies-to-watch-q3-2025/', 'published': 'Mon, 07 Jul 2025 04:01:20 GMT', 'summary': 'Analysts at Bloomberg In

In [ ]:
financial_rss_feeds = [
    "https://finance.yahoo.com/news/rssindex",
    "https://feeds.finance.yahoo.com/rss/2.0/headline?s=AAPL,MSFT,TSLA,GOOGL,AMZN&region=US&lang=en-US",
    "https://feeds.bloomberg.com/businessweek/news.rss",
    "https://fortune.com/feed",
    "https://fortune.com/section/finance/feed",
    "https://seekingalpha.com/feed.xml",
    "http://rss.cnn.com/rss/money_markets.rss",
    "http://rss.cnn.com/rss/money_news_economy.rss",
    "http://rss.cnn.com/rss/money_news_companies.rss",
    "https://fool.com/a/feeds/partner/google/",
    "http://247wallst.com/feed",
    "https://www.marketbeat.com/feed",
    "https://www.kiplinger.com/feed/all"
]

In [ ]:
@patch
def get_articles_filtered(self:RSScraper, limit=100):
    articles = self.get_articles(limit)
    return [a for a in articles if a['summary'] and a['summary'].strip()]

In [ ]:
finance_scraper = RSScraper(financial_rss_feeds)

In [ ]:
L(finance_scraper.get_articles_filtered())

(#8) [{'title': 'Startups Pivot From SEO to AI Visibility', 'link': 'https://finance.yahoo.com/news/startups-pivot-seo-ai-visibility-223737632.html?.tsrc=rss', 'published': 'Mon, 07 Jul 2025 22:37:37 +0000', 'summary': 'Athena, Profound and Scrunch lead the charge in AI search tuning'},{'title': 'Prime Day to Shatter Records with $23.8 Billion in Sales', 'link': 'https://finance.yahoo.com/news/prime-day-shatter-records-23-223326092.html?.tsrc=rss', 'published': 'Mon, 07 Jul 2025 22:33:26 +0000', 'summary': 'Adobe forecasts 28% growth as shoppers flood mobile and AI channels'},{'title': 'Meta lures Apple’s top AI executive in latest talent coup, Bloomberg reports', 'link': 'https://finance.yahoo.com/news/meta-lures-apple-top-ai-223253900.html?.tsrc=rss', 'published': 'Mon, 07 Jul 2025 22:32:53 +0000', 'summary': 'Investing.com -- Apple Inc (NASDAQ:AAPL). has lost a key figure in its artificial intelligence efforts to Meta Platforms Inc (NASDAQ:META)., as the battle for elite AI talent i

In [ ]:
L(finance_scraper)

(#1) [<__main__.RSScraper object at 0x7070281dbc80>]

In [ ]:
@patch
def get_articles(self:RSScraper, limit=50):
    articles = []
    per_feed = max(1, limit // len(self.feeds))
    for feed_url in self.feeds:
        try:
            feed = self.fetch_feed(feed_url)
            for entry in feed.entries[:per_feed]: articles.append(self._parse_entry(entry))
        except: continue
    return articles[:limit]

In [ ]:
finance_scraper = RSScraper(financial_rss_feeds)
L(finance_scraper.get_articles_filtered())

(#59) [{'title': 'Startups Pivot From SEO to AI Visibility', 'link': 'https://finance.yahoo.com/news/startups-pivot-seo-ai-visibility-223737632.html?.tsrc=rss', 'published': 'Mon, 07 Jul 2025 22:37:37 +0000', 'summary': 'Athena, Profound and Scrunch lead the charge in AI search tuning'},{'title': 'Prime Day to Shatter Records with $23.8 Billion in Sales', 'link': 'https://finance.yahoo.com/news/prime-day-shatter-records-23-223326092.html?.tsrc=rss', 'published': 'Mon, 07 Jul 2025 22:33:26 +0000', 'summary': 'Adobe forecasts 28% growth as shoppers flood mobile and AI channels'},{'title': 'Meta lures Apple’s top AI executive in latest talent coup, Bloomberg reports', 'link': 'https://finance.yahoo.com/news/meta-lures-apple-top-ai-223253900.html?.tsrc=rss', 'published': 'Mon, 07 Jul 2025 22:32:53 +0000', 'summary': 'Investing.com -- Apple Inc (NASDAQ:AAPL). has lost a key figure in its artificial intelligence efforts to Meta Platforms Inc (NASDAQ:META)., as the battle for elite AI talent 

In [ ]:
finance_scraper.get_articles_filtered()[0:5]

[{'title': 'Startups Pivot From SEO to AI Visibility',
  'link': 'https://finance.yahoo.com/news/startups-pivot-seo-ai-visibility-223737632.html?.tsrc=rss',
  'published': 'Mon, 07 Jul 2025 22:37:37 +0000',
  'summary': 'Athena, Profound and Scrunch lead the charge in AI search tuning'},
 {'title': 'Prime Day to Shatter Records with $23.8 Billion in Sales',
  'link': 'https://finance.yahoo.com/news/prime-day-shatter-records-23-223326092.html?.tsrc=rss',
  'published': 'Mon, 07 Jul 2025 22:33:26 +0000',
  'summary': 'Adobe forecasts 28% growth as shoppers flood mobile and AI channels'},
 {'title': 'Meta lures Apple’s top AI executive in latest talent coup, Bloomberg reports',
  'link': 'https://finance.yahoo.com/news/meta-lures-apple-top-ai-223253900.html?.tsrc=rss',
  'published': 'Mon, 07 Jul 2025 22:32:53 +0000',
  'summary': 'Investing.com -- Apple Inc (NASDAQ:AAPL). has lost a key figure in its artificial intelligence efforts to Meta Platforms Inc (NASDAQ:META)., as the battle for 

In [ ]:
finance_scraper = RSScraper(financial_rss_feeds)
finance_prioritizer = NewsPrioritizer(finance_scraper, finance_profiles)
finance_prioritizer.show_dashboard_ai()


1. PRIORITY 1/10: Startups Pivot From SEO to AI Visibility
   Published: Mon, 07 Jul 2025 22:37:37 +0000
   Impact: Microsoft (1/10, sentiment: 0)
   Impact: Amazon (1/10, sentiment: 0)
   Link: https://finance.yahoo.com/news/startups-pivot-seo-ai-visibility-223737632.html?.tsrc=rss

2. PRIORITY 1/10: ‘Telecom Is the New Tequila’: Behind the Celebrity Wireless Boom
   Published: Mon, 07 Jul 2025 14:23:47 GMT
   Impact: Tesla (1/10, sentiment: 0)
   Link: https://www.bloomberg.com/news/features/2025-07-07/why-the-trump-brothers-ryan-reynolds-became-telecom-entrepreneurs

3. PRIORITY 1/10: Richard ‘Dick’ Eastland, Camp Mystic’s 74-year-old owner, died while leading campers to safety during the devastating floods
   Published: Tue, 08 Jul 2025 01:03:00 +0000
   Impact: Tesla (1/10, sentiment: 0)
   Link: https://fortune.com/2025/07/07/richard-dick-eastland-camp-mystic-owner-director-death-texas-flooding/


In [ ]:
test_articles = [
    dict(title="Tesla Reports Record Q3 Deliveries, Cybertruck Production Ramps Up", 
         summary="Tesla delivered 500,000 vehicles in Q3, beating analyst expectations. Cybertruck production increased 300% with new Gigafactory capacity coming online.",
         published="Mon, 07 Jul 2025 10:00:00 +0000", link="https://example.com/tesla-q3"),
    
    dict(title="Apple iPhone 16 Sales Disappoint, Foxconn Cuts Taiwan Production", 
         summary="Apple's latest iPhone 16 sales fell short of expectations, leading assembler Foxconn to reduce production at its Taiwan facilities by 25%.",
         published="Mon, 07 Jul 2025 11:00:00 +0000", link="https://example.com/apple-iphone"),
    
    dict(title="Microsoft Azure AI Services Drive 40% Revenue Growth", 
         summary="Microsoft's cloud division posted massive gains as Azure AI and artificial intelligence services attracted enterprise customers, boosting quarterly revenue.",
         published="Mon, 07 Jul 2025 12:00:00 +0000", link="https://example.com/msft-azure"),
    
    dict(title="Amazon Prime Day Breaks Records, Robotics Division Expands", 
         summary="Amazon's Prime Day generated $15B in sales while the company announced major expansion of its robotics operations and AI-powered logistics.",
         published="Mon, 07 Jul 2025 13:00:00 +0000", link="https://example.com/amazon-prime"),
    
    dict(title="Google Nuclear Fusion Partnership Powers Virginia Data Centers", 
         summary="Google's Alphabet announced a breakthrough nuclear fusion deal to power its Virginia data centers, with lawmakers supporting the clean energy initiative.",
         published="Mon, 07 Jul 2025 14:00:00 +0000", link="https://example.com/google-nuclear")
]

In [ ]:
test_scraper = RSScraper([])
test_scraper.get_articles = lambda limit=50: test_articles
test_prioritizer = NewsPrioritizer(test_scraper, finance_profiles)
test_prioritizer.show_dashboard_ai()


1. PRIORITY 10/10: Tesla Reports Record Q3 Deliveries, Cybertruck Production Ramps Up
   Published: Mon, 07 Jul 2025 10:00:00 +0000
   Impact: Tesla (10/10, sentiment: 7)
   Impact: Netflix (1/10, sentiment: 0)
   Link: https://example.com/tesla-q3

2. PRIORITY 10/10: Apple iPhone 16 Sales Disappoint, Foxconn Cuts Taiwan Production
   Published: Mon, 07 Jul 2025 11:00:00 +0000
   Impact: Apple (10/10, sentiment: -6)
   Impact: Tesla (1/10, sentiment: 0)
   Impact: Microsoft (1/10, sentiment: 0)
   Impact: Amazon (1/10, sentiment: 0)
   Link: https://example.com/apple-iphone

3. PRIORITY 10/10: Microsoft Azure AI Services Drive 40% Revenue Growth
   Published: Mon, 07 Jul 2025 12:00:00 +0000
   Impact: Microsoft (10/10, sentiment: 7)
   Impact: Tesla (1/10, sentiment: 0)
   Impact: Amazon (1/10, sentiment: 0)
   Link: https://example.com/msft-azure

4. PRIORITY 10/10: Amazon Prime Day Breaks Records, Robotics Division Expands
   Published: Mon, 07 Jul 2025 13:00:00 +0000
   Impact: Ama

Let's put everything working in one cell which I can copy-paste and test. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

```python
import numpy as np, torch, pickle, feedparser, requests, sqlite3, re
from datetime import datetime
from pathlib import Path
from typing import Union, List, Dict, Tuple, Optional, Callable, Set
from dataclasses import dataclass
from fastcore.utils import *
from claudette import *

@dataclass
class RiskProfile:
    name: str
    ticker: Optional[str] = None
    industry: Optional[str] = None
    gics_code: Optional[str] = None
    risk_factors: Set[str] = None
    
    def __post_init__(self):
        if self.risk_factors is None: self.risk_factors = set()
        self.risk_factors.add(self.name.lower())
        if self.ticker: self.risk_factors.add(self.ticker.lower())
        if self.industry: self.risk_factors.add(self.industry.lower())

class RSScraper:
    def __init__(self, feeds): self.feeds = feeds if isinstance(feeds, list) else [feeds]
    def fetch_feed(self, url): return feedparser.parse(url)
    def get_articles(self, limit=50):
        articles = []
        per_feed = max(1, limit // len(self.feeds)) if self.feeds else 0
        for feed_url in self.feeds:
            try:
                feed = self.fetch_feed(feed_url)
                for entry in feed.entries[:per_feed]: articles.append(self._parse_entry(entry))
            except: continue
        return articles[:limit]
    def _parse_entry(self, entry): 
        summary = entry.get('summary', '') or entry.get('description', '')
        return dict(title=entry.get('title', ''), link=entry.get('link', ''), published=entry.get('published', ''), summary=summary)
    def get_articles_filtered(self, limit=100):
        articles = self.get_articles(limit)
        return [a for a in articles if a['summary'] and a['summary'].strip()]
    def analyze_sentiment_ai(self, text, risk_profile=None):
        sp = """You are a financial risk analyst. Analyze sentiment and risk impact on a scale of -10 to +10 where:
        -10: Extremely negative (major crisis, bankruptcy, massive losses)
        -5: Negative (earnings miss, regulatory issues, downgrades)
        0: Neutral (routine news, mixed signals)
        +5: Positive (earnings beat, upgrades, partnerships)
        +10: Extremely positive (breakthrough products, major acquisitions)
        
        Consider: market sentiment, fundamental impact, volatility implications, sector effects.
        Respond with only the number."""
        
        context = f"Risk factors: {', '.join(risk_profile.risk_factors) if risk_profile else 'general market'}"
        prompt = f"{context}\n\nNews: {text[:800]}"
        
        try:
            chat = Chat(models[1], sp=sp)
            result = chat(prompt)
            score = int(result.content[0].text.strip())
            return max(-10, min(10, score))
        except: return 0
    def score_risk_impact_comprehensive(self, article, risk_profiles):
        text = f"{article['title']} {article['summary']}"
        risk_scores = []
        
        for profile in risk_profiles:
            base_score = 0
            text_lower = text.lower()
            
            if profile.name.lower() in text_lower: base_score += 5
            if profile.ticker and profile.ticker.lower() in text_lower: base_score += 3
            for factor in profile.risk_factors:
                if factor in text_lower: base_score += 1
            
            if base_score > 0:
                sentiment_score = self.analyze_sentiment_ai(text, profile)
                risk_contribution = abs(sentiment_score) * 0.1
                final_score = min(int(base_score * (1 + risk_contribution)), 10)
                if final_score > 0: risk_scores.append((profile, final_score, sentiment_score))
        
        return sorted(risk_scores, key=lambda x: x[1], reverse=True)

class NewsPrioritizer:
    def __init__(self, scraper, profiles): self.scraper,self.profiles = scraper,profiles
    def get_prioritized_news_ai(self, limit=10):
        articles = self.scraper.get_articles_filtered(limit)
        prioritized = []
        for article in articles:
            scores = self.scraper.score_risk_impact_comprehensive(article, self.profiles)
            max_score = scores[0][1] if scores else 0
            prioritized.append((article, max_score, scores))
        return sorted(prioritized, key=lambda x: x[1], reverse=True)
    def show_dashboard_ai(self, limit=5):
        news = self.get_prioritized_news_ai(limit)
        for i, (article, score, matches) in enumerate(news):
            if score > 0:
                print(f"\n{i+1}. PRIORITY {score}/10: {article['title']}")
                print(f"   Published: {article['published']}")
                for profile, pscore, sentiment in matches: print(f"   Impact: {profile.name} ({pscore}/10, sentiment: {sentiment})")
                print(f"   Link: {article['link']}")

financial_rss_feeds = ["https://finance.yahoo.com/news/rssindex", "https://feeds.finance.yahoo.com/rss/2.0/headline?s=AAPL,MSFT,TSLA,GOOGL,AMZN&region=US&lang=en-US", "https://feeds.bloomberg.com/businessweek/news.rss", "https://fortune.com/feed", "https://fortune.com/section/finance/feed", "https://seekingalpha.com/feed.xml", "http://rss.cnn.com/rss/money_markets.rss", "http://rss.cnn.com/rss/money_news_economy.rss", "http://rss.cnn.com/rss/money_news_companies.rss", "https://fool.com/a/feeds/partner/google/", "http://247wallst.com/feed", "https://www.marketbeat.com/feed", "https://www.kiplinger.com/feed/all"]

finance_profiles = [
    RiskProfile("Tesla", "TSLA", "Automobiles", "251020", {"tesla", "tsla", "elon musk", "cybertruck", "electric vehicle", "ev", "autopilot", "robotaxi", "delivery", "production", "gigafactory"}),
    RiskProfile("Microsoft", "MSFT", "Software", "451030", {"microsoft", "msft", "ai", "artificial intelligence", "cloud", "azure", "pakistan", "operations", "diagnostic", "mai-dxo"}),
    RiskProfile("Apple", "AAPL", "Technology Hardware", "452020", {"apple", "aapl", "iphone", "earnings", "foxconn", "taiwan", "assembler"}),
    RiskProfile("Amazon", "AMZN", "Internet Retail", "255020", {"amazon", "amzn", "prime day", "shopping", "exec", "insider", "ai", "shein", "temu", "robotics"}),
    RiskProfile("Google/Alphabet", "GOOGL", "Internet Software", "451010", {"google", "googl", "alphabet", "nuclear", "fusion", "power", "virginia", "energy", "lawmakers"}),
    RiskProfile("Netflix", "NFLX", "Entertainment", "254010", {"netflix", "nflx", "streaming", "trillion", "market cap", "analyst", "upgrade"})
]

finance_scraper = RSScraper(financial_rss_feeds)
finance_prioritizer = NewsPrioritizer(finance_scraper, finance_profiles)
finance_prioritizer.show_dashboard_ai(5)
```



In [ ]:
import numpy as np, torch, pickle, feedparser, requests, sqlite3, re
from datetime import datetime
from pathlib import Path
from typing import Union, List, Dict, Tuple, Optional, Callable, Set
from dataclasses import dataclass
from fastcore.utils import *
from claudette import *

@dataclass
class RiskProfile:
    name: str
    ticker: Optional[str] = None
    industry: Optional[str] = None
    gics_code: Optional[str] = None
    risk_factors: Set[str] = None
    
    def __post_init__(self):
        if self.risk_factors is None: self.risk_factors = set()
        self.risk_factors.add(self.name.lower())
        if self.ticker: self.risk_factors.add(self.ticker.lower())
        if self.industry: self.risk_factors.add(self.industry.lower())

class RSScraper:
    def __init__(self, feeds): self.feeds = feeds if isinstance(feeds, list) else [feeds]
    def fetch_feed(self, url): return feedparser.parse(url)
    def get_articles(self, limit=50):
        articles = []
        per_feed = max(1, limit // len(self.feeds)) if self.feeds else 0
        for feed_url in self.feeds:
            try:
                feed = self.fetch_feed(feed_url)
                for entry in feed.entries[:per_feed]: articles.append(self._parse_entry(entry))
            except: continue
        return articles[:limit]
    def _parse_entry(self, entry): 
        summary = entry.get('summary', '') or entry.get('description', '')
        return dict(title=entry.get('title', ''), link=entry.get('link', ''), published=entry.get('published', ''), summary=summary)
    def get_articles_filtered(self, limit=100):
        articles = self.get_articles(limit)
        return [a for a in articles if a['summary'] and a['summary'].strip()]
    def analyze_sentiment_ai(self, text, risk_profile=None):
        sp = """You are a financial risk analyst. Analyze sentiment and risk impact on a scale of -10 to +10 where:
        -10: Extremely negative (major crisis, bankruptcy, massive losses)
        -5: Negative (earnings miss, regulatory issues, downgrades)
        0: Neutral (routine news, mixed signals)
        +5: Positive (earnings beat, upgrades, partnerships)
        +10: Extremely positive (breakthrough products, major acquisitions)
        
        Consider: market sentiment, fundamental impact, volatility implications, sector effects.
        Respond with only the number."""
        
        context = f"Risk factors: {', '.join(risk_profile.risk_factors) if risk_profile else 'general market'}"
        prompt = f"{context}\n\nNews: {text[:800]}"
        
        try:
            chat = Chat(models[1], sp=sp)
            result = chat(prompt)
            score = int(result.content[0].text.strip())
            return max(-10, min(10, score))
        except: return 0
    def score_risk_impact_comprehensive(self, article, risk_profiles):
        text = f"{article['title']} {article['summary']}"
        risk_scores = []
        
        for profile in risk_profiles:
            base_score = 0
            text_lower = text.lower()
            
            if profile.name.lower() in text_lower: base_score += 5
            if profile.ticker and profile.ticker.lower() in text_lower: base_score += 3
            for factor in profile.risk_factors:
                if factor in text_lower: base_score += 1
            
            if base_score > 0:
                sentiment_score = self.analyze_sentiment_ai(text, profile)
                risk_contribution = abs(sentiment_score) * 0.1
                final_score = min(int(base_score * (1 + risk_contribution)), 10)
                if final_score > 0: risk_scores.append((profile, final_score, sentiment_score))
        
        return sorted(risk_scores, key=lambda x: x[1], reverse=True)

class NewsPrioritizer:
    def __init__(self, scraper, profiles): self.scraper,self.profiles = scraper,profiles
    def get_prioritized_news_ai(self, limit=10):
        articles = self.scraper.get_articles_filtered(limit)
        prioritized = []
        for article in articles:
            scores = self.scraper.score_risk_impact_comprehensive(article, self.profiles)
            max_score = scores[0][1] if scores else 0
            prioritized.append((article, max_score, scores))
        return sorted(prioritized, key=lambda x: x[1], reverse=True)
    def show_dashboard_ai(self, limit=5):
        news = self.get_prioritized_news_ai(limit)
        for i, (article, score, matches) in enumerate(news):
            if score > 0:
                print(f"\n{i+1}. PRIORITY {score}/10: {article['title']}")
                print(f"   Published: {article['published']}")
                for profile, pscore, sentiment in matches: print(f"   Impact: {profile.name} ({pscore}/10, sentiment: {sentiment})")
                print(f"   Link: {article['link']}")

financial_rss_feeds = ["https://finance.yahoo.com/news/rssindex", "https://feeds.finance.yahoo.com/rss/2.0/headline?s=AAPL,MSFT,TSLA,GOOGL,AMZN&region=US&lang=en-US", "https://feeds.bloomberg.com/businessweek/news.rss", "https://fortune.com/feed", "https://fortune.com/section/finance/feed", "https://seekingalpha.com/feed.xml", "http://rss.cnn.com/rss/money_markets.rss", "http://rss.cnn.com/rss/money_news_economy.rss", "http://rss.cnn.com/rss/money_news_companies.rss", "https://fool.com/a/feeds/partner/google/", "http://247wallst.com/feed", "https://www.marketbeat.com/feed", "https://www.kiplinger.com/feed/all"]

finance_profiles = [
    RiskProfile("Tesla", "TSLA", "Automobiles", "251020", {"tesla", "tsla", "elon musk", "cybertruck", "electric vehicle", "ev", "autopilot", "robotaxi", "delivery", "production", "gigafactory"}),
    RiskProfile("Microsoft", "MSFT", "Software", "451030", {"microsoft", "msft", "ai", "artificial intelligence", "cloud", "azure", "pakistan", "operations", "diagnostic", "mai-dxo"}),
    RiskProfile("Apple", "AAPL", "Technology Hardware", "452020", {"apple", "aapl", "iphone", "earnings", "foxconn", "taiwan", "assembler"}),
    RiskProfile("Amazon", "AMZN", "Internet Retail", "255020", {"amazon", "amzn", "prime day", "shopping", "exec", "insider", "ai", "shein", "temu", "robotics"}),
    RiskProfile("Google/Alphabet", "GOOGL", "Internet Software", "451010", {"google", "googl", "alphabet", "nuclear", "fusion", "power", "virginia", "energy", "lawmakers"}),
    RiskProfile("Netflix", "NFLX", "Entertainment", "254010", {"netflix", "nflx", "streaming", "trillion", "market cap", "analyst", "upgrade"})
]

# finance_scraper = RSScraper(financial_rss_feeds)
# finance_prioritizer = NewsPrioritizer(finance_scraper, finance_profiles)
# finance_prioritizer.show_dashboard_ai(5)

test_articles = [
    dict(title="Tesla Reports Record Q3 Deliveries, Cybertruck Production Ramps Up", 
         summary="Tesla delivered 500,000 vehicles in Q3, beating analyst expectations. Cybertruck production increased 300% with new Gigafactory capacity coming online.",
         published="Mon, 07 Jul 2025 10:00:00 +0000", link="https://example.com/tesla-q3"),
    
    dict(title="Apple iPhone 16 Sales Disappoint, Foxconn Cuts Taiwan Production", 
         summary="Apple's latest iPhone 16 sales fell short of expectations, leading assembler Foxconn to reduce production at its Taiwan facilities by 25%.",
         published="Mon, 07 Jul 2025 11:00:00 +0000", link="https://example.com/apple-iphone"),
    
    dict(title="Microsoft Azure AI Services Drive 40% Revenue Growth", 
         summary="Microsoft's cloud division posted massive gains as Azure AI and artificial intelligence services attracted enterprise customers, boosting quarterly revenue.",
         published="Mon, 07 Jul 2025 12:00:00 +0000", link="https://example.com/msft-azure"),
    
    dict(title="Amazon Prime Day Breaks Records, Robotics Division Expands", 
         summary="Amazon's Prime Day generated $15B in sales while the company announced major expansion of its robotics operations and AI-powered logistics.",
         published="Mon, 07 Jul 2025 13:00:00 +0000", link="https://example.com/amazon-prime"),
    
    dict(title="Google Nuclear Fusion Partnership Powers Virginia Data Centers", 
         summary="Google's Alphabet announced a breakthrough nuclear fusion deal to power its Virginia data centers, with lawmakers supporting the clean energy initiative.",
         published="Mon, 07 Jul 2025 14:00:00 +0000", link="https://example.com/google-nuclear")
]
test_scraper = RSScraper([])
test_scraper.get_articles = lambda limit=50: test_articles
test_prioritizer = NewsPrioritizer(test_scraper, finance_profiles)
test_prioritizer.show_dashboard_ai()

<!-- """MVP Portfolio Risk Profile Creator - fast.ai style""" -->
```
from fastcore.all import *
import yfinance as yf
import numpy as np
import pandas as pd
from typing import Dict, List, Tuple
import warnings; warnings.filterwarnings('ignore')

# Core functions following fast.ai style
def get_returns(tickers, period="1y"): 
    "Download price data and calculate returns for `tickers`"
    data = yf.download(tickers, period=period, progress=False)['Adj Close']
    return data.pct_change().dropna() if len(tickers) > 1 else data.pct_change().dropna().to_frame()

def calc_port_vol(rets, wts): 
    "Calculate portfolio volatility from returns and weights"
    cov = rets.cov() * 252
    return np.sqrt(wts.T @ cov @ wts)

def calc_risk_contrib(rets, wts, port_vol):
    "Calculate risk contribution for each position"
    cov = rets.cov() * 252
    port_cov = cov @ wts
    return L((wts[i] * port_cov.iloc[i] / port_vol**2, ticker) 
             for i,ticker in enumerate(rets.columns))

def get_sector_info(tickers):
    "Get sector mapping for tickers - simplified version"
    sectors = {}
    for ticker in tickers:
        try: 
            info = yf.Ticker(ticker).info
            sectors[ticker] = info.get('sector', 'Unknown')
        except: sectors[ticker] = 'Unknown'
    return sectors

# Main portfolio risk profile class
class PortfolioRisk:
    "MVP Portfolio Risk Profile using fast.ai style"
    
    def __init__(self, holdings:Dict[str,float], period:str="1y"):
        """
        Create risk profile from holdings dict
        holdings: {'AAPL': 0.3, 'MSFT': 0.25, 'GOOGL': 0.2, ...}  # weights
        """
        store_attr('holdings,period')
        self.tickers,self.wts = self._parse_holdings()
        self.rets = get_returns(self.tickers, period)
        self.sectors = get_sector_info(self.tickers)
        self._calc_risk_metrics()
    
    def _parse_holdings(self):
        "Extract tickers and weights as arrays"
        tickers = L(self.holdings.keys())
        wts = np.array(L(self.holdings.values()))
        assert abs(wts.sum() - 1.0) < 0.01, f"Weights sum to {wts.sum():.3f}, should be ~1.0"
        return tickers,wts
    
    def _calc_risk_metrics(self):
        "Calculate all risk metrics"
        self.port_vol = calc_port_vol(self.rets, self.wts) 
        risk_contribs = calc_risk_contrib(self.rets, self.wts, self.port_vol)
        
        # Store risk contributions 
        self.risk_contrib = {ticker: contrib for contrib,ticker in risk_contribs}
        
        # Calculate individual stock volatilities
        self.stock_vols = (self.rets.std() * np.sqrt(252)).to_dict()
        
        # Sector aggregation
        self.sector_risk = self._calc_sector_risk()
        
        # Risk concentration (risk_contrib / weight)
        self.risk_conc = {t: self.risk_contrib[t]/self.holdings[t] for t in self.tickers}
    
    def _calc_sector_risk(self):
        "Aggregate risk by sector"
        sector_risk = {}
        for ticker in self.tickers:
            sector = self.sectors[ticker] 
            if sector not in sector_risk: sector_risk[sector] = 0
            sector_risk[sector] += self.risk_contrib[ticker]
        return sector_risk
    
    @property 
    def profile(self):
        "Return complete risk profile dict"
        return {
            'total_risk': self.port_vol * 100,
            'risk_contributions': {t: self.risk_contrib[t]*100 for t in self.tickers},
            'weights': {t: self.holdings[t]*100 for t in self.tickers}, 
            'risk_concentration': self.risk_conc,
            'stock_volatilities': {t: v*100 for t,v in self.stock_vols.items()},
            'sector_risk': {s: r*100 for s,r in self.sector_risk.items()}
        }
    
    def top_risks(self, n:int=5):
        "Show top N risk contributors"
        sorted_risks = sorted(self.risk_contrib.items(), key=lambda x: abs(x[1]), reverse=True)
        return L(sorted_risks[:n])
    
    def risk_summary(self):
        "Print concise risk summary"
        print(f"Portfolio Risk: {self.port_vol*100:.1f}%")
        print(f"\nTop Risk Contributors:")
        for ticker,risk in self.top_risks():
            wt,conc = self.holdings[ticker]*100, self.risk_conc[ticker]
            print(f"  {ticker:5} {wt:5.1f}% wt → {risk*100:5.1f}% risk (conc: {conc:.1f}x)")
        
        print(f"\nSector Breakdown:")  
        for sector,risk in sorted(self.sector_risk.items(), key=lambda x: x[1], reverse=True):
            if risk > 0.01: print(f"  {sector:12} {risk*100:5.1f}%")

# Simplified news scoring using risk profile            
def score_news_risk(news_impact:float, affected_ticker:str, risk_profile:PortfolioRisk):
    "Score news impact using risk-weighted approach"
    if affected_ticker not in risk_profile.holdings: return 0
    
    # Risk-weighted impact vs simple weight-weighted
    risk_wt = risk_profile.risk_contrib[affected_ticker] 
    concentration = risk_profile.risk_conc[affected_ticker]
    
    # News impact scaled by risk contribution and concentration
    return news_impact * risk_wt * min(concentration, 3.0)  # Cap extreme concentration

# Quick test function
def test_portfolio_risk():
    "Test the portfolio risk calculator"
    
    # Sample portfolio
    holdings = {'AAPL': 0.3, 'MSFT': 0.25, 'GOOGL': 0.2, 'TSLA': 0.15, 'NVDA': 0.1}
    
    # Create risk profile  
    pf = PortfolioRisk(holdings)
    pf.risk_summary()
    
    # Test news scoring
    news_scores = [
        ("AAPL earnings miss", -2.5, 'AAPL'),
        ("TSLA recall announced", -1.8, 'TSLA'), 
        ("Tech regulation fears", -1.2, 'GOOGL')
    ]
    
    print(f"\nNews Risk Scores:")
    for desc,impact,ticker in news_scores:
        score = score_news_risk(impact, ticker, pf)
        print(f"  {desc:20} {score:6.3f}")

if __name__ == "__main__": test_portfolio_risk()
```